In [149]:
import urllib.request
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('C:\\Users\\ext_fantmiraudal\\Downloads\\new_data\\data.csv')

In [3]:
# Extractor de imagenes desde data
i=808
for row in data['image'][i:]:
    without_image = row.find('[]')
    if without_image == -1:
        index1 = row.index('url')
        index2 = row.index('png')
        image_url = row[(index1+7):(index2+3)]
        #blabla
        index3 = row.index('/cl-images/')
        # setting filename and image URL
        #filename = 'C:\\Users\\ext_fantmiraudal\\Downloads\\new_data\\sample_' + str(i) + '.png'
        filename = 'C:\\Users\\ext_fantmiraudal\\Downloads\\new_data\\' + row[(index3+10):(index2+3)]
        # calling urlretrieve function to get resource
        urllib.request.urlretrieve(image_url, filename)
        i+=1
    else:
        i+=1    

In [4]:
data['image'][0:]

0      {\n  "image": [{\n    "url": "https://prdadess...
1      {\n  "image": [{\n    "url": "https://prdadess...
2      {\n  "image": [{\n    "url": "https://prdadess...
3      {\n  "image": [{\n    "url": "https://prdadess...
4      {\n  "image": [{\n    "url": "https://prdadess...
                             ...                        
995    {\n  "image": [{\n    "url": "https://prdadess...
996    {\n  "image": [{\n    "url": "https://prdadess...
997    {\n  "image": [{\n    "url": "https://prdadess...
998    {\n  "image": [{\n    "url": "https://prdadess...
999    {\n  "image": [{\n    "url": "https://prdadess...
Name: image, Length: 1000, dtype: object

In [ ]:
# Prueba de modelo con dataset
df_base = pd.DataFrame(columns = ['url', 'Prob. Producto', 'Producto', 'Prob. Sin rostro cliente',\
                                  'Sin rostro cliente', 'Contexto', 'Prob. Etiqueta producto', 'Etiqueta producto',\

                                  'Prob. Numero domicilio', 'Numero domicilio', 'Score'])

jason_path_api= "clave2\\tc-sc-bi-bigdata-corp-tsod-dev-739359c1cf2a.json"
jason_path_automl="key\\tc-sc-bi-bigdata-corp-tsod-dev-82c15beedd2a.json"
i=0
for row in data['image'][i:]:
    without_image = row.find('[]')
    if without_image == -1:
        index1 = row.index('url')
        index2 = row.index('png')
        image_url = row[(index1+7):(index2+3)]
        df1 = pd.DataFrame(columns = ['url'])  
        #aplicar modelo que retornara df2
        df2 = full_model(image_url,jason_path_api, jason_path_automl)
        df3 = pd.concat((df1,df2), axis =1)
        df_base = pd.concat((df_base,df3), ignore_index= True)
        i+=1
    else:
        i+=1  
        
#descargar df_base como excel
#se especifica nombre del archivo
file_name = 'resultados_modelo.xlsx'
  
#guardar excel
df_base.to_excel(file_name)
print('Archivo excel se exportó correctamente')

In [135]:
#importar excel resultados_modelo y etiquetado_manual
resultados_modelos = 'resultados_modelo.xlsx'
etiquetado = 'etiquetado_manual.xlsx'
resultados_modelos = pd.read_excel(resultados_modelos)
etiquetado = pd.read_excel(etiquetado)

In [147]:
resultados_modelos['Producto']

0      0
1      1
2      1
3      1
4      1
      ..
871    0
872    0
873    0
874    1
875    1
Name: Producto, Length: 876, dtype: int64

In [152]:
#calculo accuracy modelo para cada criterio menos contexto
#crear listas vacias
aciertos_producto = np.empty(0)
aciertos_sin_rostro = np.empty(0)
aciertos_etiqueta = np.empty(0)
aciertos_domicilio = np.empty(0)
#recorrer archivos excel
for indice in resultados_modelos.index:
    #restar => los 0 son los aciertos
    aciertos_producto = np.append(aciertos_producto, resultados_modelos['Producto'][indice]-etiquetado['Producto'][indice]) 
    aciertos_sin_rostro = np.append(aciertos_sin_rostro,(1-resultados_modelos['Sin rostro cliente'][indice])-etiquetado['Aparece Rostro'][indice]) 
    aciertos_etiqueta = np.append(aciertos_etiqueta, resultados_modelos['Etiqueta producto'][indice]-etiquetado['Etiqueta producto'][indice]) 
    aciertos_domicilio = np.append(aciertos_domicilio, resultados_modelos['Numero domicilio'][indice]-etiquetado['Numero domicilio'][indice])

In [159]:
#accuracies
acc_producto = np.count_nonzero(aciertos_producto == 0)/np.shape(aciertos_producto)[0]
print('acc. producto: ', acc_producto)
acc_sin_rostro = np.count_nonzero(aciertos_sin_rostro == 0)/np.shape(aciertos_sin_rostro)[0]
print('acc. sin_rostro: ', acc_sin_rostro)
acc_etiqueta = np.count_nonzero(aciertos_etiqueta == 0)/np.shape(aciertos_etiqueta)[0]
print('acc. etiqueta: ', acc_etiqueta)
acc_domicilio = np.count_nonzero(aciertos_domicilio == 0)/np.shape(aciertos_domicilio)[0]
print('acc. domicilio: ', acc_domicilio)

acc. producto:  0.5753424657534246
acc. sin_rostro:  0.8287671232876712
acc. etiqueta:  0.591324200913242
acc. domicilio:  0.7442922374429224
